### The purpose of this notebook is to complete a data cleaning workflow from start to finish in order to validate the core functionality our package

#### TO DO:
- Add in complete PubChem data
- Write PubChem function
- Organize code modules & tests
- Clean up/finish writing tests
- Write main script wrapper function


In [1]:
# imports

from core import *
from cpd_info import *
from mol_sim_copy import *
from pubchem_client import *

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

## Step 1

#### Read in master dataframe



In [2]:
# read in the master_df

master_df = pd.read_csv('../datasets/MASTER_DF.csv') 
# master_df = master_df[master_df['reacts'] == 1.0]    # select only the reactive enzyme - compound pairs
# master_df = master_df[['entry', 'PubChem', 'SMILES']]
print(master_df.shape)
master_df.head()

(6879, 24)


,entry,product,reacts,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,1.8.99.5,C00094,1.0,3394,OS(=O)O,<rdkit.Chem.rdchem.Mol object at 0x1ac9b8a210>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.000000,1,0,...,0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,0.0,82.080
1,1.13.11.18,C00094,1.0,3394,OS(=O)O,<rdkit.Chem.rdchem.Mol object at 0x1ac9b8a580>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.511007,1,0,...,0,0.0,2.0,3.0,0.0,0.0,1.0,0.0,0.0,82.080
2,1.8.99.5,C00283,1.0,3578,S,<rdkit.Chem.rdchem.Mol object at 0x1ac9b8ac10>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.000000,1,0,...,0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,34.083
3,2.8.1.2,C00283,1.0,3578,S,<rdkit.Chem.rdchem.Mol object at 0x1ac9b8a2b0>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.241667,0,1,...,0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,34.083
4,4.4.1.28,C00283,1.0,3578,S,<rdkit.Chem.rdchem.Mol object at 0x1ac9b8a120>,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,0.294605,0,0,...,0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,34.083


## Step 2 
#### Get query SMILES string & pair query compound with each unique enzyme in the master DataFrame
Example: PubChem SID 3480

In [3]:
# function to query the SMILES string and append new pairs to the master dataframe

def pair_query_compound(master_df, enzyme_col, pubchem_col, smiles_col, pubchem_sid):
    """
    pair_query_compound_with_enzymes() queries pubchem to get a SMILES string from an input pubchem_sid,
        then pairs that query compound with each unique enzyme id in the master dataframe
        
    Args:
        master_df (pandas.DataFrame): master dataframe containing enzyme ids
        enzyme_col (str): column containing enzyme id
        pubchem_col (str): column containing pubchem sid
        smiles_col (str): column containing SMILES string
        pubchem_sid (str): query PubChem sid
        
    Returns:
        pandas.DataFrame: with rows added to include query compound
    """
    master_df = master_df[[enzyme_col, pubchem_col, smiles_col]]
    new_pairs = []
    smiles, _ = sid_to_smiles(pubchem_sid)
    if len(smiles) == 0:
        raise 'query compound SMILES string could not be retrieved'
    else:
        pass
    unique_enzymes = master_df[enzyme_col].unique().tolist()
    for enzyme in unique_enzymes:
        pair = {enzyme_col:enzyme, pubchem_col:pubchem_sid, smiles_col:smiles}
        new_pairs.append(pair)
    new_paris_df = pd.DataFrame(new_pairs)
    output_df = pd.concat((master_df, new_paris_df), axis=0, sort=False)
    return output_df


In [4]:
# pair_query_compound_with_enzymes()

updated_df = pair_query_compound(master_df, 'entry', 'PubChem', 'SMILES', '3480')
print(updated_df.shape)
updated_df.head()

(7395, 3)


,entry,PubChem,SMILES
0,1.8.99.5,3394,OS(=O)O
1,1.13.11.18,3394,OS(=O)O
2,1.8.99.5,3578,S
3,2.8.1.2,3578,S
4,4.4.1.28,3578,S


## Step 3
#### Calculate molecular distances between products of the same enzyme
Positive and negative data must be calculated separately because the calculated molecular distance is an average of the distances between all products of any one enzyme. The model is based on the fact that we expect these distances to be closer for reactive enzyme-product pairs than for non-reactive enzyme-product pairs

In [5]:
# calculate_dist()
# positive data

distance_df = calculate_dist(updated_df)
print(distance_df.shape)
distance_df.head()

(7395, 6)


,entry,PubChem,SMILES,Mol,Fingerprint,dist
0,1.8.99.5,3394,OS(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261F45...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.284869
1,1.13.11.18,3394,OS(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261F45...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0.335205
2,1.8.99.5,3578,S,<rdkit.Chem.rdchem.Mol object at 0x00000261F45...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.284869
3,2.8.1.2,3578,S,<rdkit.Chem.rdchem.Mol object at 0x00000261F45...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.293565
4,4.4.1.28,3578,S,<rdkit.Chem.rdchem.Mol object at 0x00000261F45...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.353979


## Step 4
#### Get dummy variables to represent enzyme class
We expect that many enzyme properties could be predictive features for this model. Enzyme class should encapsulate many of these features at a high level.

In [6]:
# remove any rows that are not the query compound

reduced_df = distance_df[distance_df['PubChem'] == '3480']

In [7]:
# binarize_enzyme_class()

query_df = binarize_enzyme_class(reduced_df, 'entry')
query_df = query_df.reset_index(drop=True)
print(query_df.shape)
query_df.head()

(516, 13)


C:\Users\ellie\Documents\Data Science Classes\metamoles\code\core.py:282: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataframe['enzyme_class'] = [row[column][0] for _, row in dataframe.iterrows()]


,entry,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,enzyme_class_5,enzyme_class_6,enzyme_class_7
0,1.8.99.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.284869,1,0,0,0,0,0,0
1,1.13.11.18,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.335205,1,0,0,0,0,0,0
2,2.8.1.2,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.293565,0,1,0,0,0,0,0
3,4.4.1.28,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.353979,0,0,0,1,0,0,0
4,3.13.1.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.301190,0,0,1,0,0,0,0


## Step 5
#### Add in compound features with RDKit
This step uses the RDKit packages to generate descriptive features of the reaction product compounds

In [8]:
query_df = create_cpd_info(query_df)
print(query_df.shape)
query_df.head()

(516, 22)


,entry,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,enzyme_class_3,enzyme_class_4,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,1.8.99.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.284869,1,0,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
1,1.13.11.18,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.335205,1,0,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
2,2.8.1.2,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.293565,0,1,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
3,4.4.1.28,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.353979,0,0,0,1,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
4,3.13.1.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.301190,0,0,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123


## Step 6
#### Re-Instantiate model

In [10]:
feature_df = master_df[['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU']]
features = np.array(feature_df) #shape balance array for regression
reactions = list(master_df['reacts'])
feature_train, feature_test, reaction_train, reaction_test = train_test_split(features, reactions,
                                                  test_size=0.20, random_state=42)
model_1 = linear_model.LogisticRegression(solver='liblinear', penalty='l1', random_state=1, class_weight='balanced')
model_1.fit(feature_train, np.ravel(reaction_train))

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l1', random_state=1,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [25]:
# Test the model
score = model_1.score(feature_test, reaction_test)
print('The model score is ' + str(round(score*100, 2)) + '%.')

The model score is 92.01%.


## Step 7 
#### Use model to predict reactivity of pairs

In [11]:
# select query features

query_feat_df = query_df[['dist', 'enzyme_class_1', 'enzyme_class_2', 'enzyme_class_3',
       'enzyme_class_4', 'enzyme_class_5', 'enzyme_class_6', 'enzyme_class_7',
       'n_O', 'n_N', 'n_P', 'n_S', 'n_X', 'DoU']]

In [12]:
predictions = model_1.predict(query_feat_df) # change me to the data you want to predict based on

pred = model_1.predict_proba(query_feat_df)

prediction_values = pd.DataFrame(pred)
model_descriptive_df = pd.DataFrame()
model_descriptive_df['0']=prediction_values[0]
model_descriptive_df['1']=prediction_values[1]

In [13]:
prediction_df = pd.merge(model_descriptive_df, query_df, left_index=True, right_index=True) 
print(prediction_df.shape)
prediction_df.head()

(516, 24)


,0,1,entry,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
0,0.951881,0.048119,1.8.99.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.284869,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
1,0.879565,0.120435,1.13.11.18,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.335205,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
2,0.948478,0.051522,2.8.1.2,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.293565,0,1,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
3,0.656857,0.343143,4.4.1.28,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.353979,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
4,0.951829,0.048171,3.13.1.5,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.301190,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123


In [14]:
prediction_df = prediction_df.sort_values(by=['1'], ascending=False)
prediction_df.head()

,0,1,entry,PubChem,SMILES,Mol,Fingerprint,dist,enzyme_class_1,enzyme_class_2,...,enzyme_class_7,n_C,n_H,n_O,n_N,n_P,n_S,n_X,DoU,MW
359,0.202246,0.797754,1.1.1.187,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.504968,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
365,0.250458,0.749542,1.3.1.115,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.491018,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
319,0.251098,0.748902,1.14.14.50,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.490846,1,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
174,0.252059,0.747941,4.1.1.96,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.441723,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123
254,0.265738,0.734262,4.2.1.165,3480,C1=CC=C(C=C1)C(=O)O,<rdkit.Chem.rdchem.Mol object at 0x00000261FB5...,"[0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, ...",0.438121,0,0,...,0,7.0,6.0,2.0,0.0,0.0,0.0,0.0,5.0,122.123


## Out of curiosity:
#### Examine average molecular distance distributions for negative and positive data
On first glance, it appears that our hypothesis is correct in that the distributions of average molecular distances are qualitatively different between the positive and negative datasets

In [ ]:
# look at distributions of distances for positive and negative data

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

pos = sns.distplot(distance_df['dist'], bins=50, kde=False, ax=axes[0])
axes[0].set_title('positive data avg. mol. dist.')

neg = sns.distplot(distance_df['dist'], bins=50, kde=False, ax=axes[1])
axes[1].set_title('negative data avg. mol. dist.')

for axis in axes:
    axis.set_xlim([0.0, 1.0])